Import Necessary Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot
import plotly.graph_objs as go
import plotly.express as px
import math
import folium

Loading Data

In [2]:
# df = pd.read_csv('C:/Users/GAME/covid19-in-india/date_wise_data.csv')
df1 = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
df2 = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
df3 = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
# df = df.drop(df.index[-1])
df1 = df1.set_index('Country/Region')
df2 = df2.set_index('Country/Region')
df3 = df3.set_index('Country/Region')

india1 = df1.loc['India']
india1 = india1.astype('float64')
india1 = india1.drop(['Lat','Long'])
india2 = df2.loc['India']
india2 = india2.astype('float64')
india2 = india2.drop(['Lat','Long'])
india3 = df3.loc['India']
india3 = india3.astype('float64')
india3 = india3.drop(['Lat','Long'])

# india2

In [3]:
india_df = pd.DataFrame(india1)
india_df = india_df.drop(['Province/State'])
india_df.insert(1, 'rec', india2)
india_df.insert(2, 'deaths', india3)
india_df.insert(3, 'Active', india1-india2-india3)
india_df = india_df.reset_index().rename(columns = {'index':'Date', 'India':'Confirmed', 'rec':'Cured', 'deaths':'Deaths'})#.set_index('Date')
df = india_df.copy()
df.Date = pd.to_datetime(df.Date)#, dayfirst=True)
df

,Date,Confirmed,Cured,Deaths,Active
0,2020-01-22,0.0,0.0,0.0,0.0
1,2020-01-23,0.0,0.0,0.0,0.0
2,2020-01-24,0.0,0.0,0.0,0.0
3,2020-01-25,0.0,0.0,0.0,0.0
4,2020-01-26,0.0,0.0,0.0,0.0
...,...,...,...,...,...
119,2020-05-20,112028.0,45422.0,3434.0,63172.0
120,2020-05-21,118226.0,48553.0,3584.0,66089.0
121,2020-05-22,124794.0,51824.0,3726.0,69244.0
122,2020-05-23,131423.0,54385.0,3868.0,73170.0


In [5]:
ac= 76820
rvd = 57692
dth = 4024
fig = go.Figure(data=[go.Pie(labels=['Active Cases','Cured','Death'],
                             values= [ac,rvd,dth],hole =.3)])
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=['#263fa3', '#2fcc41','#cc3c2f'], line=dict(color='#FFFFFF', width=2)))
fig.update_layout(title_text='Current Situation in India',plot_bgcolor='rgb(275, 270, 273)')
fig.show()

In [28]:
p_df = pd.read_csv('C:/Users/GAME/covid19-in-india/IndividualDetails.csv')

tempa = p_df[['current_status','gender']].dropna()
actv = tempa[tempa['current_status'] == 'Hospitalized']
a_f = len(actv[actv['gender'] == 'F'])
a_m = len(actv[actv['gender'] == 'M'])
t = a_m + a_f
fig = go.Figure(data=[go.Pie(labels=['Male','Female'],
                             values= [math.ceil(a_m/t*100), int(a_f/t*100)],hole =.3)])
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=['#263fa3', '#d461bf'], line=dict(color='#FFFFFF', width=2)))
fig.update_layout(title_text='Gender wise Active Cases',plot_bgcolor='rgb(275, 270, 273)')
fig.show()


In [29]:
tempr = p_df[['current_status','gender']].dropna()
rcvd = tempr[tempr['current_status'] == 'Recovered']
r_f = len(rcvd[rcvd['gender'] == 'F'])
r_m = len(rcvd[rcvd['gender'] == 'M'])
fig = go.Figure(data=[go.Pie(labels=['Male','Female'],
                             values= [r_m,r_f],hole =.3)])
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=['#263fa3', '#d461bf'], line=dict(color='#FFFFFF', width=2)))
fig.update_layout(title_text='Gender wise Recovered Cases(Numbers are inaccurate due to missing values)',
plot_bgcolor='rgb(275, 270, 273)')
fig.show()

In [40]:
temp = p_df[['status_change_date','age','current_status','gender','detected_state']].dropna()
rec = temp[temp['current_status'] == 'Recovered'].drop('current_status',axis =1).sort_values('status_change_date',ascending=True)
rec_x = rec['age'].astype(int)
rec_y = rec['gender']
fig = px.histogram(x=rec_x,color =rec_y,orientation = 'v',
                   title='Age wise Recovered cases in Male and Female (Numbers are inaccurate due to missing values)')
fig.update_layout(barmode='stack')
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)',yaxis_title='Deaths',xaxis_title='Age Group')
fig.show()

In [57]:
test1 = pd.read_csv('ICMRTestingDetails.csv')
test1.DateTime = pd.to_datetime(test1.DateTime, dayfirst=True)
test1 = test1.ffill(axis = 0)
test1.head()

,SNo,DateTime,TotalSamplesTested,TotalIndividualsTested,TotalPositiveCases
0,1,2020-03-13 18:00:00,6500.0,5900.0,78.0
1,2,2020-03-18 18:00:00,13125.0,12235.0,150.0
2,3,2020-03-19 18:00:00,14175.0,13285.0,182.0
3,4,2020-03-20 18:00:00,15404.0,14514.0,236.0
4,5,2020-03-21 18:00:00,16911.0,16021.0,315.0


In [56]:
import plotly.express as px
fig = px.bar(test1, x="DateTime", y="TotalSamplesTested", barmode='group',height=500,color = "TotalSamplesTested",
             orientation = 'v',
            color_discrete_sequence = px.colors.sequential.Plasma_r)
fig.update_layout(title_text='Number of COVID-19 test conducted everyday',plot_bgcolor='rgb(275, 270, 273)')
fig.update_layout(barmode='stack')
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)',yaxis_title='Tests',xaxis_title='Date')
fig.show()

In [60]:
fig = go.Figure(data=[
go.Bar(name='Tested', x=test1['DateTime'], y=test1['TotalSamplesTested'],marker_color='#2fcc41'),
go.Bar(name='Positive', x=test1['DateTime'], y=test1['TotalPositiveCases'],marker_color='#FF0000')])
fig.update_layout(barmode='stack',width=500, height=600)
fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.update_layout(title_text='Number of people tested and positive among them',
                  plot_bgcolor='rgb(275, 270, 273)')
fig.show()

In [7]:
perday = df[df['Confirmed'] >= 10].groupby(['Date'])['Confirmed'].sum().reset_index().sort_values('Confirmed',ascending = True)
perday['New Daily Cases'] = perday['Confirmed'].sub(perday['Confirmed'].shift())
perday['New Daily Cases'].iloc[0] = perday['Confirmed'].iloc[0]
perday['New Daily Cases'] = perday['New Daily Cases'].astype(int)
# New COVID-19 cases reported daily in India
import plotly.express as px
fig = px.bar(perday, x="Date", y="New Daily Cases", barmode='group',height=500)
fig.update_layout(title_text='New COVID-19 cases reported daily in India',plot_bgcolor='rgb(275, 270, 273)')
fig.show()

D:\anaconda\new_anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [8]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['Date'], y=df['Confirmed'],
                    mode='lines+markers',marker_color='blue',name='Total Cases'))
fig.add_trace(go.Scatter(x=df['Date'],y=df['Active'], 
                mode='lines+markers',marker_color='purple',name='Active'))
fig.add_trace(go.Scatter(x=df['Date'], y=df['Cured'],
                mode='lines+markers',marker_color='green',name='Recovered'))
fig.add_trace(go.Scatter(x=df['Date'], y=df['Deaths'], 
                mode='lines+markers',marker_color='red',name='Deaths'))
fig.update_layout(title_text='Trend of Weekly Coronavirus Cases in India',plot_bgcolor='rgb(275, 270, 273)',width=1000, height=600)
fig.show()

In [80]:
cnf = '#263fa3' # confirmed - blue
act = '#fe9801' # active case - yellow
rec = '#21bf73' # recovered - green
dth = '#de260d' # death - red
tmp = df.melt(id_vars="Date",value_vars=['Deaths','Cured' ,'Active','Confirmed'],
                 var_name='Case',value_name='Count')
fig = px.area(tmp, x="Date", y="Count",color='Case',
              title='Trend Over Weeks',color_discrete_sequence = [dth,rec,act,cnf])
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)',width=550, height=600)
fig.show()

In [12]:
df_india_data = pd.read_csv('C:/Users/GAME/covid19-in-india/covid_19_india.csv')
df_india_data = df_india_data[['Date', 'State/UnionTerritory', 'Cured', 'Deaths', 'Confirmed']]
df_india_data.Date = pd.to_datetime(df_india_data.Date, dayfirst=True)
df_india_data

,Date,State/UnionTerritory,Cured,Deaths,Confirmed
0,2020-01-30,Kerala,0,0,1
1,2020-01-31,Kerala,0,0,1
2,2020-02-01,Kerala,0,0,2
3,2020-02-02,Kerala,0,0,3
4,2020-02-03,Kerala,0,0,3
...,...,...,...,...,...
1831,2020-05-11,Telengana,750,30,1196
1832,2020-05-11,Tripura,2,0,150
1833,2020-05-11,Uttarakhand,46,1,68
1834,2020-05-11,Uttar Pradesh,1653,74,3467


In [93]:
spread = df_india_data.groupby(['Date', 'State/UnionTerritory'])['Confirmed'].sum().reset_index().sort_values('Confirmed', ascending=True)
fig = px.area(spread, x="Date", y="Confirmed",color='State/UnionTerritory',title='State Wise Spread over time',height=500,
              color_discrete_sequence = ex.colors.cyclical.Edge)
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)',width=700, height=600)

In [10]:
temp = df.copy()
temp['Mortality Rate'] = temp['Deaths']/temp['Confirmed']*100
fig = go.Figure()
fig.add_trace(go.Scatter(x=temp['Date'], y=temp['Mortality Rate'],mode='lines+markers',marker_color='red'))
fig.update_layout(title_text = 'Trend of Mortality Rate of India')
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)',width=1000, height=600)
fig.show()

In [97]:
agegroup = pd.read_csv('C:/Users/GAME/AgeGroupDetails.csv')
fig = go.Figure()
fig.add_trace(go.Scatter(x=agegroup['AgeGroup'],y=agegroup['TotalCases'],line_shape='spline',fill='tonexty',fillcolor = 'orange')) 
fig.update_layout(title="Age wise Confirmed Case Trend in India",yaxis_title="Total Number of cases",xaxis_title="Age Group")
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)',width=600,height=600)
fig.show()

In [13]:
temp = df_india_data.sort_values('Confirmed', ascending=True)
fig = go.Figure(data=[
go.Bar(name='Confirmed', y=temp['State/UnionTerritory'], x=temp['Confirmed'], 
       orientation='h',marker_color='#0f5dbd'),
    go.Bar(name='Cured', y=temp['State/UnionTerritory'], x=temp['Cured'], 
       orientation='h',marker_color='#319146'),
go.Bar(name='Death', y=temp['State/UnionTerritory'], x=temp['Deaths'], 
       orientation='h',marker_color='red')])
fig.update_layout(barmode='stack',width=600, height=800)
#fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.update_layout(title_text='Confirmed Cases,Cured,Deaths in Different States of India',
                  plot_bgcolor='rgb(275, 270, 273)')
fig.show()